# Case study of automated inference of qualitative models of ecological system: a planktonic ecosystem

We apply the software BoNesis for the inference of qualitative models of ecological systems from time series observations.

Binarized time series data come from (Gaucherel et al, 2024: https://peercommunityjournal.org/articles/10.24072/pcjournal.417/), which studied the following ecological network):

![](https://codimd.math.cnrs.fr/uploads/upload_2722375ce10eb40eb04cb5baeb3b47af.png)


In [1]:
import bonesis
import pandas as pd
from tqdm import tqdm
from IPython.display import display, Markdown
from boeco import *

## Observations

The case study includes two time series recorded at two marine stations on successive weeks during winter and spring 2012 to reconstruct the network trajectories and their environment over
time (Masclaux et al., 2014). The time series have been binarized in (Gaucherel et al, 2024).

For instance, from station A, the time series consists of 3 successive regimes:

![](https://codimd.math.cnrs.fr/uploads/upload_e080a142d6c2ff5c5b61d98fae9b2242.png)

We consider `Bact` and `Nit` as variables reflecting the state of environment and season. For the inference, we will interpret the data as follows:
```
While (Bact=0 and Nit=1):
    A1 regime is stable
While (Bact=1 and Nit=1):
    There must exist a trajectory from A1 to A2, and
    A2 regime is stable
```
and so on.

We specify the time series from (Gaucherel et al, 2024):

In [2]:
species = ["Bact", "PicoP", "NanoP", "MicrP", "Proto", 
           "MicrZ", "MesoZ", "Nit"]
inputs = ["Nit", "Bact"]

In [3]:
_zero = {a: 0 for a in species}
_one = {a: 1 for a in species}
data = {
    "A1": _zero | {"Nit": 1},
    "A2": _one | {"NanoP": 0, "Proto": 0},
    "A3": _one | {"NanoP": 0, "MicrZ": 0, "Nit": 0},
    
    "B1": _zero | {"Nit": 1},
    "B2": _one | {"NanoP": 0, "MicrZ": 0},
    "B3": _one | {"NanoP": 0, "Proto": 0},
    "B4": _one | {"MicrP": 0, "Proto": 0, "Nit": 0},

    "T1": _zero | { "Nit": 1},
    "T2": _zero | {"NanoP": 1, "MicrP": 1, "Nit": 1},
    "T3": _one | {"Bact": 0, "PicoP": 0, "Proto": 0},
    "T4": _one | {"MicrP": 0, "Nit": 0},
    "T5": _one | {"NanoP": 0, "MicrP": 0, "MesoZ": 0, "Nit": 0},
    "T6": _zero | {"Bact": 1, "PicoP": 1},
}
data_df = pd.DataFrame(data)
data_df

,A1,A2,A3,B1,B2,B3,B4,T1,T2,T3,T4,T5,T6
Bact,0,1,1,0,1,1,1,0,0,0,1,1,1
PicoP,0,1,1,0,1,1,1,0,0,0,1,1,1
NanoP,0,0,0,0,0,0,1,0,1,1,1,0,0
MicrP,0,1,1,0,1,1,0,0,1,1,0,0,0
Proto,0,0,1,0,1,0,0,0,0,0,1,1,0
MicrZ,0,1,0,0,0,1,1,0,0,1,1,1,0
MesoZ,0,1,1,0,1,1,1,0,0,1,1,0,0
Nit,1,1,0,1,1,1,0,1,1,1,0,0,0


## Prior influence graph
The domain of admissible Boolean networks is specified from a prior influence graph.
From the complete graph, we remove influences that violate basic constraints related to the type and size of species.

In [4]:
pkn = bonesis.InfluenceGraph.complete(species, loops=True)
pkn.remove_edges_from(list(pkn.in_edges(inputs)))

def make_pos(d):
    d["sign"] = 1; d["label"] = "+"
def make_neg(d):
    d["sign"] = -1; d["label"] = "-"


# Bact does not eat anybody, we remove negative interactions from them
for _,_,d in pkn.out_edges("Bact", data=True):
    make_pos(d)

# Proto does not eat anybody but Bact, PicoP, NanoP
for _, b, d in pkn.out_edges("Proto", data=True):
    if b.endswith("Z") or b == "MicrP":
        make_pos(d)
for b, _, d in pkn.in_edges("Proto", data=True):
    if b.endswith("Z") or b == "MicroP":
        make_neg(d)

# Phytoplankton does not eat Zooplankton, nor themselves, but they may benefit from preferential
for a, b, d in pkn.out_edges(["PicoP", "NanoP", "MicrP"], data=True):
    if b.endswith("Z") or b.endswith("P") or (a != "MicrP" and b == "Proto"):
        make_pos(d)
for b, a, d in pkn.in_edges(["PicoP", "NanoP", "MicrP"], data=True):
    if b.endswith("Z") or (a != "MicrP" and b == "Proto"):
        make_neg(d)

# MicroZ cannot eat MesoZ 
for _, b, d in pkn.out_edges("MicrZ", data=True):
    if b == "MesoZ":
        make_pos(d)
for b, _, d in pkn.in_edges("MicrZ", data=True):
    if b == "MesoZ":
        make_neg(d)

# Meso cannot eat Bact
pkn.remove_edges_from([("MesoZ","Bact"),("Bact","MesoZ")])
pkn.remove_edges_from([("MicrP","Proto"),("Proto","MicrP")])

pkn

# computing graph layout...


## Expected dynamical properties

We specify the expected dynamical properties following our interpretation of time series data.

In [6]:
def get_cond(name):
    return dict(data_df.loc[inputs, name])
def station_A_constraints(bo):
    with bo.action(get_cond("A1")):
        bo.fixed(~bo.obs("A1"))
    with bo.action(get_cond("A2")):
        +bo.obs("A1") >= ~bo.obs("A2")
        bo.fixed(~bo.obs("A2"))
    with bo.action(get_cond("A3")):
        +bo.obs("A2") >= ~bo.obs("A3")
        bo.fixed(~bo.obs("A3"))

def station_B_constraints(bo):
    with bo.action(get_cond("B1")):
        bo.fixed(~bo.obs("B1"))
    with bo.action(get_cond("B3")):
        ~bo.obs("B1") >= ~bo.obs("B2") >= ~bo.obs("B3")
        bo.fixed(~bo.obs("B3"))
    with bo.action(get_cond("B4")):
        with bo.scope_reachability(monotone=False):
            ~bo.obs("B3") >= ~bo.obs("B4")
        bo.fixed(~bo.obs("B4"))

def theoretical_constraints(bo):
    for t in ["T1", "T2", "T3", "T4", "T5", "T6"]:
        with bo.action(get_cond(t)):
            bo.fixed(~bo.obs(t))

In [7]:
dom = bonesis.InfluenceGraph(pkn, maxclause=16)
bo = bonesis.BoNesis(dom, data)
with bo.scope_reachability(monotone=True):
    station_A_constraints(bo)
    station_B_constraints(bo)
    theoretical_constraints(bo)

assert bo.is_satisfiable()

## Inferred influence graphs

Because of the few observations, multiple candidate models possess the specified dynamical properties.

We demonstrate here below how to use bonesis to analyze the full sets of solution.

### Intersection between all models

We rely on `clingo` capability to compute intersection between all solutions to extract the influences that are shared by all the Boolean networks compatible with the prior influence graph and the specified dynamical properties.

In [8]:
%%time
view = bonesis.InfluenceGraphView(bo, clingo_options=("--enum-mode=cautious",))
for core_ig in view:
    pass
core_ig

Grounding...done in 0.6s
CPU times: user 38.6 s, sys: 48 ms, total: 38.7 s
Wall time: 38.7 s
# computing graph layout...


### Union of all models

Then, we enumerate the influence graphs that are used by at least one solution Boolean network, and display the union of them by weighting the influences with their frequency.

In [9]:
from collections import Counter

In [10]:
%%time

dom = bonesis.InfluenceGraph(pkn, maxclause=8, canonic=True)
bo = bonesis.BoNesis(dom, data)
with bo.scope_reachability(monotone=True):
    station_A_constraints(bo)
    station_B_constraints(bo)
    theoretical_constraints(bo)

c = Counter()
view = bonesis.InfluenceGraphView(bo, solutions="subset-minimal", progress=tqdm, maxclause=16)
nb = 0
for ig in tqdm(view):
    c.update(Counter(((a,b,d["label"]) for a,b,d in ig.edges(data=True))))
    nb += 1
nb

0it [00:00, ?it/s]

Grounding...done in 0.2s


2400it [00:17, 134.93it/s]


CPU times: user 17.7 s, sys: 60.1 ms, total: 17.7 s
Wall time: 17.8 s


2400

In [11]:
g = nx.MultiDiGraph()
g.add_edges_from((a,b,{"sign": {"+":1,"-":-1}.get(d,0), "label": f"{d}", "penwidth": 2*w/nb}) for (a,b,d),w in c.items())
g

# computing graph layout...
